In [1]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data_after_cleansing_normalized.csv')

In [3]:
df.head()

,Tweet,HS,normalized_tweet,clean_tweet
0,- disaat semua cowok berusaha melacak perhatia...,1,- di saat semua pria berusaha melacak perhatia...,di saat semua pria berusaha melacak perhatian...
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,RT USER: USER siapa telat memberi tau elu?eda...,rt user user siapa telat memberi tau eluedan s...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,"41. Kadang aku berfikir, kenapa aku tetap perc...",kadang aku berfikir kenapa aku tetap percaya ...
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,user user aku itu akunnku tau matamu sipit tap...
4,USER USER Kaum cebong kapir udah keliatan dong...,1,USER USER Kaum kecebong kafir sudah kelihatan ...,user user kaum kecebong kafir sudah kelihatan ...


In [4]:
df.shape

(18396, 4)

In [5]:
df['HS'].value_counts()

HS
0    10947
1     7449
Name: count, dtype: int64

In [6]:
#check maximum length of word in sentence 
max_sen = df['clean_tweet'].str.split().str.len().max()

In [7]:
max_sen

62

**Split the Data**

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, log_loss
from sklearn.metrics import classification_report, confusion_matrix

In [9]:
x_train, x_test, y_train, y_test = train_test_split(df['clean_tweet'], df['HS'], test_size = 0.2, random_state = 42,stratify=df['HS'])

In [10]:
len(x_train)

14716

In [11]:
 x_train.str.split().str.len().max()

62

**Remove Stopword and Tokenization**

In [12]:
#Define Stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('indonesian')
# Get a list of stop words in the English language
stop_words = set(stopwords.words('indonesian'))

# Display the top 20 stop words
list(stop_words)[:20]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gilan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gilan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading indonesian: Package 'indonesian' not found
[nltk_data]     in index


['sekaligus',
 'pertanyakan',
 'lebih',
 'jika',
 'kesampaian',
 'beginilah',
 'kemungkinannya',
 'sendirian',
 'ibaratnya',
 'meyakinkan',
 'sekalipun',
 'bagaimana',
 'menyampaikan',
 'akhir',
 'dimulailah',
 'beginikah',
 'ditunjuki',
 'saya',
 'ditunjuk',
 'menjelaskan']

In [13]:
#tokenization
word_token=[word_tokenize(i) for i in x_train]

In [14]:
word_token

[['some',
  'berpikir',
  'doesnt',
  'add',
  'up',
  'from',
  'the',
  'present',
  'racial',
  'situation',
  'in',
  'melayu',
  'islam',
  'cina',
  'bukan',
  'islam',
  'kafir',
  'biadap',
  'dapigs',
  'menterbalikkan',
  'jalur',
  'gemilang',
  'komunis',
  'since',
  'when',
  'race',
  'and',
  'religion',
  'became',
  'the',
  'sama',
  'thing',
  'make',
  'up',
  'your',
  'mind',
  'which',
  'card',
  'are',
  'you',
  'playing',
  'race',
  'orang',
  'religion',
  'pictwittercomftbmnagsh'],
 ['because',
  'he',
  'cant',
  'actualy',
  'provide',
  'a',
  'satisfactory',
  'answer',
  'debatcapres',
  'prabowo',
  'tolak',
  'jawab',
  'soal',
  'ham'],
 ['user',
  'ini',
  'aku',
  'salut',
  'dari',
  'rejim',
  'skrg',
  'konsisten',
  'kenceng',
  'di',
  'media',
  'massa',
  'kicep',
  'di',
  'dunia',
  'nyata',
  'ntab'],
 ['user',
  'user',
  'user',
  'celengnya',
  'teriak',
  'teriak',
  'kecebong',
  'tolol',
  'padahal',
  'celeng',
  'lebih',
  'dun

In [15]:
len(word_token)

14716

In [16]:
# Remove stopwords from each sublist in word_token
filtered_tokens_train = [[word for word in sublist if word not in stop_words] for sublist in word_token]

# Display the first sublist of filtered tokens after removing stopwords
print(filtered_tokens_train[0])

['some', 'berpikir', 'doesnt', 'add', 'up', 'from', 'the', 'present', 'racial', 'situation', 'in', 'melayu', 'islam', 'cina', 'islam', 'kafir', 'biadap', 'dapigs', 'menterbalikkan', 'jalur', 'gemilang', 'komunis', 'since', 'when', 'race', 'and', 'religion', 'became', 'the', 'thing', 'make', 'up', 'your', 'mind', 'which', 'card', 'are', 'you', 'playing', 'race', 'orang', 'religion', 'pictwittercomftbmnagsh']


In [17]:
print(filtered_tokens_train[1])

['because', 'he', 'cant', 'actualy', 'provide', 'a', 'satisfactory', 'answer', 'debatcapres', 'prabowo', 'tolak', 'ham']


**Vectorization Word2Vec - Skipgram**

In [24]:
import gensim
from gensim.models import Word2Vec 
model_skipgram = gensim.models.Word2Vec(filtered_tokens_train, min_count = 1,vector_size = 64, window = 3, sg=1) 

In [25]:
vocabulary_skipgram = model_skipgram.wv.index_to_key
word_vec_dict={}
for word in vocabulary_skipgram:
    word_vec_dict[word]=model_skipgram.wv.get_vector(word)
print("The no of key-value pairs : ",len(word_vec_dict)) # should come equal to vocab size

The no of key-value pairs :  32815


In [26]:
word_vec_dict

{'user': array([ 0.54706717, -0.40005162,  0.6242414 ,  0.49705878, -0.34379703,
        -0.52056867,  0.568814  , -0.27175274, -0.7938128 , -0.24556686,
         0.4470143 , -0.47839412, -0.17310318,  0.01842991, -0.18693353,
         0.41216984, -0.5617102 , -0.2816284 , -0.4902487 ,  0.5268871 ,
         0.5063543 ,  1.0434823 ,  0.9997121 , -0.56273127, -0.06524556,
         0.5891143 , -0.622978  ,  0.14355412, -0.16020398, -0.48512378,
        -0.3197176 , -0.03416171, -0.40246928, -0.63297594, -0.40637937,
        -0.13167405,  0.45850673,  0.05236329,  0.718565  ,  0.17246613,
         0.14620396,  0.45097905, -0.2684631 , -0.546253  ,  0.15399328,
        -0.2954866 , -0.15379389,  0.15040243,  0.23805477,  0.5579381 ,
         0.2666706 , -0.08323471, -0.05132806,  0.77873355,  0.61052024,
         0.07067642,  0.36138478, -0.6975297 , -0.4296941 ,  0.80601037,
        -0.5090356 , -0.30661443, -0.30664742,  0.02805568], dtype=float32),
 'the': array([ 0.21335319, -0.27316222

In [39]:
max_sen_len= max_sen # max lenght of word in a sentence
vocab_size =50000  #ideally it should be len(tok.word_index) + 1  or total no of words in data in this case = 4750, but to handle number of data which not appear in train, for example in test, make the size higher
embed_dim=64 # embedding dimension as choosen in word2vec constructor

In [40]:
from keras.preprocessing.text import one_hot,Tokenizer
tok = Tokenizer()
tok.fit_on_texts(filtered_tokens_train)
vocab_size = vocab_size
encd_rev = tok.texts_to_sequences(filtered_tokens_train)

In [41]:
# now creating the embedding matrix
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
    embed_vector=word_vec_dict.get(word) #mapping the vector to word in our skipgram dictionary
    if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
        embed_matrix[i]=embed_vector
  # if word is not found then embed_vector corressponding to that vector will stay zero.

**Preparing the Data for Embedding Layer**

In [42]:
encd_rev

[[253,
  348,
  642,
  6358,
  105,
  66,
  2,
  4324,
  6359,
  2418,
  12,
  787,
  20,
  29,
  20,
  43,
  6360,
  12564,
  12565,
  3292,
  12566,
  50,
  2661,
  90,
  1513,
  5,
  678,
  2944,
  2,
  307,
  207,
  105,
  36,
  812,
  379,
  3293,
  26,
  7,
  5135,
  1513,
  9,
  678,
  12567],
 [96, 70, 193, 490, 3734, 8, 12568, 2419, 8326, 58, 1064, 660],
 [1, 2662, 465, 1250, 1920, 12569, 315, 1921, 12570, 212, 693, 8327],
 [1,
  1,
  1,
  12571,
  308,
  308,
  75,
  202,
  1790,
  137,
  52,
  2945,
  2420,
  248,
  16,
  12572,
  328,
  6361,
  12573,
  2080,
  283,
  5136,
  56,
  328,
  8328,
  3294],
 [12574, 8329, 4325, 8330, 903, 72, 5137, 20, 8331, 752, 5138],
 [2946,
  3735,
  2241,
  2663,
  4326,
  4327,
  4328,
  4329,
  15,
  1,
  2947,
  87,
  941,
  27,
  1115,
  1298,
  1422,
  1922,
  1923,
  4330],
 [22, 39, 2, 13, 12575, 12576],
 [3736,
  832,
  56,
  2948,
  120,
  1065,
  537,
  5,
  5139,
  36,
  2421,
  49,
  128,
  39,
  208,
  443,
  201,
  45,
  19,


In [43]:
vocab_size

50000

In [44]:
from keras.utils import pad_sequences
# now padding to have a maximum length of 64
pad_rev= pad_sequences(encd_rev, maxlen=max_sen_len, padding='post')
pad_rev.shape   # note that we had 4749 data and we have padded each review to have  a lenght of 64 words.

(14716, 62)

In [45]:
pad_rev

array([[ 253,  348,  642, ...,    0,    0,    0],
       [  96,   70,  193, ...,    0,    0,    0],
       [   1, 2662,  465, ...,    0,    0,    0],
       ...,
       [3594,    2, 3595, ...,    0,    0,    0],
       [   1,    1,    1, ...,    0,    0,    0],
       [ 106, 8135,    6, ...,    0,    0,    0]])

**Classification**

In [46]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.initializers import Constant
from tensorflow.keras.layers import Dense, Embedding, Activation, Flatten

In [60]:
model = keras.Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_sen_len,embeddings_initializer=Constant(embed_matrix)))
model.add(layers.LSTM(64))
model.add(layers.BatchNormalization())
model.add(layers.Dense(2))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 62, 64)            3200000   
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 3,233,410
Trainable params: 3,233,282
Non-trainable params: 128
_________________________________________________________________
None


In [61]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)

In [62]:
model.fit(pad_rev, y_train, batch_size=2, epochs=2)

Epoch 1/2
7358/7358 [==============================] - 166s 22ms/step - loss: 0.6780 - accuracy: 0.5928
Epoch 2/2
7358/7358 [==============================] - 170s 23ms/step - loss: 0.6763 - accuracy: 0.5937


**Testing**

**Preparing  Test Data for Embedding Layer**

In [63]:
# Tokenize the words in the testing text data
word_token_test =[word_tokenize(i) for i in x_test]

In [64]:
word_token_test[0]

['di',
 'amerika',
 'diketahui',
 'sekitar',
 'orang',
 'setiap',
 'tahunnya',
 'pindah',
 'dari',
 'agama',
 'kristen',
 'ke',
 'agama',
 'islam']

In [65]:
# Remove stopwords from each sublist in word_token_test
filtered_tokens_test = [[word for word in sublist if word not in stop_words] for sublist in word_token_test]
# Display the first sublist of filtered tokens after removing stopwords
print(filtered_tokens_test[0])

['amerika', 'orang', 'tahunnya', 'pindah', 'agama', 'kristen', 'agama', 'islam']


In [66]:
tok.fit_on_texts(filtered_tokens_test)
encd_rev_test = tok.texts_to_sequences(filtered_tokens_test)

In [67]:
encd_rev_test

[[1212, 9, 11358, 829, 25, 66, 25, 21],
 [1, 1, 907, 1992, 85, 344, 14632, 37, 7000, 824, 14633],
 [822, 185, 2898, 2287, 23],
 [14634, 7, 24, 121, 373, 221, 37, 65],
 [1, 1207, 414, 1947, 3562, 135, 2514],
 [1, 201, 252, 14635, 1384, 9, 2368, 14636, 421, 53],
 [1, 1, 1, 14637, 3952, 574, 3523, 195, 4571, 839, 14638, 14639],
 [16, 3464, 3213, 682, 370],
 [119, 539],
 [11320, 2, 32, 10, 75, 76, 210, 969, 137, 143, 11173],
 [55,
  127,
  41,
  1579,
  40,
  28,
  948,
  1118,
  260,
  711,
  538,
  4,
  2939,
  33,
  323,
  1998,
  24,
  96],
 [31,
  57,
  2283,
  636,
  514,
  598,
  9,
  225,
  1,
  83,
  840,
  1723,
  26,
  6,
  3662,
  19,
  1446,
  32,
  96],
 [1, 1, 735, 14640, 9003, 188, 735, 320, 8960, 811, 2823],
 [115, 189, 517, 132, 10843, 1775, 589],
 [1, 6678, 14641, 498, 2233],
 [7796,
  556,
  561,
  9092,
  989,
  14642,
  14643,
  341,
  14644,
  2613,
  14645,
  210,
  25,
  355,
  14646,
  1502,
  11,
  145,
  5696],
 [1, 1, 1, 1, 7918, 279, 5509, 9280, 5509],
 [22, 9

In [68]:
from keras.utils import pad_sequences
# now padding to have a maximum length of 64
pad_rev_test= pad_sequences(encd_rev_test, maxlen=max_sen_len, padding='post')
pad_rev_test.shape   # note that we had 10 testing data and we have padded each review to have  a lenght of 64 words.

# from keras.utils import pad_sequences
# # now padding to have a maximum length of 64
# pad_rev= pad_sequences(encd_rev, maxlen=max_sen_len, padding='post')
# pad_rev.shape   # note that we had 4749 data and we have padded each review to have  a lenght of 64 words.

(14716, 62)

In [69]:
pad_rev_test

array([[ 1212,     9, 11358, ...,     0,     0,     0],
       [    1,     1,   907, ...,     0,     0,     0],
       [  822,   185,  2898, ...,     0,     0,     0],
       ...,
       [  855,  6974,   803, ...,     0,     0,     0],
       [ 4732,  6244,  3636, ...,     0,     0,     0],
       [ 1554,  3641,  7278, ...,     0,     0,     0]])

In [70]:
test_predict=model.predict(pad_rev_test)
classe_test=np.argmax(test_predict,axis=1)

115/115 [==============================] - 1s 7ms/step


In [71]:
classe_test

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [72]:
print('\nClassification Report\n')
print(classification_report(y_test, classe_test))


Classification Report

              precision    recall  f1-score   support

           0       0.60      1.00      0.75      2190
           1       0.00      0.00      0.00      1490

    accuracy                           0.60      3680
   macro avg       0.30      0.50      0.37      3680
weighted avg       0.35      0.60      0.44      3680

